In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from scipy.stats.stats import pearsonr
import pickle
import statistics

In [2]:
# load dataset
padel_desc = pd.read_csv(r'data/padel_desc.csv')
pIC50_target = pd.read_excel(r'data/pIC50_target.xlsx')

In [3]:
#kolom yang dihapus
padel_desc = padel_desc.drop(["Name"], axis=1)
pIC50_target = pIC50_target.drop(['No.'], axis=1)

padel_desc.shape, pIC50_target.shape

((134, 1875), (134, 1))

In [4]:
dataset = pd.concat([padel_desc, pIC50_target], axis=1)
dataset

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
0,0,-1.0627,1.129331,79.6556,45.508860,6,6,42,22,20,...,0.230608,0.561977,0.489222,0.523529,13.444158,47.047030,106.927129,0.459836,1.574728,5.842
1,0,-0.0693,0.004802,113.5328,61.298032,0,0,54,30,24,...,0.124809,0.554310,0.365546,0.392117,25.097284,103.740040,238.275628,0.709610,1.311973,6.292
2,0,-1.2672,1.605796,89.2215,51.164446,6,6,47,25,22,...,0.093133,0.554587,0.403332,0.402626,21.435407,63.448720,134.077300,0.764904,1.360546,5.469
3,0,-1.2656,1.601743,119.6603,64.662032,12,12,57,33,24,...,0.123379,0.544054,0.403809,0.370641,28.756514,130.576181,305.286387,0.723444,1.318503,6.310
4,0,-1.6474,2.713927,110.0384,59.971239,12,12,53,30,23,...,0.141888,0.581195,0.397869,0.322742,24.574375,101.480573,223.519513,0.700411,1.301806,6.387
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,0,0.3636,0.132205,138.9661,74.733204,17,18,62,34,28,...,0.152839,0.455611,0.377196,0.437523,29.375999,164.436509,430.436141,0.651397,1.270330,7.142
130,0,-0.3540,0.125316,124.2705,67.995239,17,18,58,35,23,...,0.364984,0.561717,0.357159,0.480373,24.895532,165.105971,384.973632,0.409651,1.399248,7.026
131,0,-0.4442,0.197314,124.3983,67.885446,17,18,58,36,22,...,0.368969,0.550420,0.363275,0.480499,24.939796,166.299510,388.900703,0.409199,1.394195,7.259
132,0,-0.8526,0.726927,130.9140,71.890825,17,18,62,37,25,...,0.336061,0.533630,0.431886,0.421620,22.937123,144.035418,372.377249,0.368102,1.387136,7.301


In [5]:
# drop kolom yang ada duplikat
dataset = dataset.loc[:,~dataset.columns.duplicated()]
dataset.shape

(134, 1876)

In [6]:
# drop kolom yang ada 0
dataset = dataset[(dataset.sum(axis=1) != 0)]
dataset.shape

(134, 1876)

In [7]:
# drop kolom yang ada nan
dataset.dropna(axis=1, inplace=True)
dataset.shape

(134, 1876)

In [8]:
dataset.isna().values.any()

False

In [9]:
train, test = train_test_split(dataset, test_size = 0.2, random_state = 45)
train.shape, test.shape
# test

((107, 1876), (27, 1876))

In [10]:
pickle.dump(train, open('data/train_bams.p','wb'))
pickle.dump(test, open('data/test_bams.p','wb'))

# 100cor

In [13]:
df_train = pickle.load(open('data/train_bams.p','rb'))
df_test = pickle.load(open('data/test_bams.p','rb'))
df_train.shape, df_test.shape

df_test.head()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,...,P2s,E1s,E2s,E3s,Ts,As,Vs,Ks,Ds,pIC50
80,0,-1.1920,1.420864,77.4597,42.316895,6,6,38,23,15,...,0.103904,0.601004,0.496041,0.387152,18.511855,47.601010,96.299403,0.762528,1.484197,5.252
104,1,-1.2779,1.633028,120.4713,62.803653,12,12,55,34,21,...,0.233677,0.518844,0.390298,0.403002,21.243280,118.290618,325.584633,0.453505,1.312144,5.367
94,0,-0.7765,0.602952,129.5342,68.990825,12,12,61,36,25,...,0.201191,0.559365,0.406162,0.375220,29.398513,164.906359,348.126053,0.638760,1.340746,5.319
71,1,-0.6371,0.405896,119.9204,63.335239,0,0,56,33,23,...,0.099910,0.560842,0.400273,0.367394,32.154591,124.491769,257.886479,0.797271,1.328508,6.666
51,0,-0.3282,0.107715,100.4111,60.724239,6,6,55,32,23,...,0.261880,0.553492,0.518677,0.491566,17.128885,85.914338,234.029145,0.340869,1.563734,8.569


In [19]:
corr = df_train.corr()
corr = corr.iloc[:-1,[-1]]
corr.sort_values("pIC50", ascending= False, inplace = True)
corr = corr.iloc[:100,:]
corr.to_csv (r'data/data_corr.csv', index = None, header=True)
corr

,pIC50
VCH-5,0.604421
C2SP3,0.554315
MATS6s,0.544128
VCH-6,0.539325
MATS6e,0.537876
...,...
ATSC0i,0.297018
maxaaN,0.296742
AATSC4v,0.295970
MATS4v,0.295904


In [20]:
cek = corr.index.values.tolist()[:100]
pickle.dump(cek, open('data/corr_bams.p','wb'))

In [21]:
# -----------REMOVE FEATURE WITH LOW CORRELATION TO Y RESPONSE---------------------
label_ = df_train.columns.values
pic50 = df_train.iloc[:,-1]
corr_y = [pearsonr(df_train.iloc[:,i],pic50) for i in range(df_train.shape[1])]
corr_y = [np.abs(corr_y[i][0]) for i in range(df_train.shape[1])]
corr_lim = 0.20
hi_corr = []
for i in range(len(corr_y)):
    if corr_y[i] > corr_lim:
        hi_corr.append(i)

corr_low = corr_y
        
# label_idx = label_[hi_corr]
# train = train.loc[:,label_idx]
# print("descriptor number after removing low correlation with target: {}".format(train.shape[1]))

C:\Users\Bams\anaconda3\lib\site-packages\scipy\stats\stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [22]:
df_corr = df_train.T
df_corr['corr'] = corr_low
df_corr

,125,74,1,110,99,33,21,38,107,116,...,65,15,120,121,68,95,32,124,131,corr
nAcid,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.213985
ALogP,0.038200,-0.797000,-0.069300,-2.577400,-0.408000,-0.514900,-0.504700,-1.560200,-0.660300,-1.969300,...,-1.195100,-0.504700,-0.150100,-0.330500,-2.091000,-0.158900,-0.408200,-0.829100,-0.444200,0.072253
ALogp2,0.001459,0.635209,0.004802,6.642991,0.166464,0.265122,0.254722,2.434224,0.435996,3.878142,...,1.428264,0.254722,0.022530,0.109230,4.372281,0.025249,0.166627,0.687407,0.197314,0.061178
AMR,138.119400,77.237500,113.532800,128.735300,117.906800,105.992200,119.532000,96.119000,133.499900,133.714600,...,80.044000,119.532000,133.147600,133.403200,119.917600,142.562800,106.067300,140.046700,124.398300,0.183939
apol,72.933239,41.352102,61.298032,70.286825,62.001653,58.637653,64.662032,59.111618,72.084411,73.599997,...,44.182067,64.662032,71.639618,71.420032,64.300032,78.271583,58.637653,75.315618,67.885446,0.026160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
As,174.085678,43.726934,103.740040,143.408571,131.000413,96.423082,116.965395,118.298132,213.453177,146.321769,...,45.677709,120.474440,161.652428,169.586341,105.897604,278.796848,96.524233,174.995325,166.299510,0.067991
Vs,385.225921,87.066620,238.275628,358.730867,332.588534,262.879539,259.314016,258.448065,494.620934,333.050590,...,103.946438,253.060199,355.246880,373.650224,242.885706,771.668789,263.097835,389.991476,388.900703,0.043498
Ks,0.674239,0.775200,0.709610,0.514894,0.377649,0.300649,0.719133,0.684823,0.702543,0.408998,...,0.450186,0.749250,0.668801,0.670822,0.720412,0.680110,0.277946,0.685353,0.409199,0.213217
Ds,1.450915,1.475112,1.311973,1.394896,1.565066,1.430683,1.430927,1.502287,1.464436,1.350846,...,1.580583,1.358829,1.393983,1.450742,1.640487,1.387634,1.408755,1.453048,1.394195,0.165392


In [23]:
# # -----------REMOVE FEATURE WITH HIGH CORRELATION TO OTHER FEATURE---------------------
# # re-calculate correlation with pic50
# label_ = train.columns.values
# corr_y = [pearsonr(train.iloc[:,i],pic50) for i in range(train.shape[1])]
# corr_y = [np.abs(corr_y[i][0]) for i in range(train.shape[1])]
# desc_num = train.shape[1]
# # calculate correlation for each descriptor
# corr_matrix = np.corrcoef(train.T)
# corr_lim = 0.80
# low_corr = np.arange(desc_num).tolist()
# tmp = np.arange(desc_num).tolist()
# for i in np.arange(desc_num):
#     tmp.remove(i)
#     for j in tmp:
#         corr_ = np.abs(corr_matrix[i,j])
#         if corr_ >= corr_lim:
#             if corr_y[i] > corr_y[j]:
#                 if j in low_corr:
#                     low_corr.remove(j)
#             else:
#                 if i in low_corr:
#                     low_corr.remove(i)
# label_idx = label_[low_corr]
# df_train = train.loc[:,label_idx]
# print("descriptor number after removing high correlation descriptor: {}".format(df_train.shape[1]))
# df_train

In [24]:
df_corr = df_corr.sort_values(by='corr', ascending=False)
df_corr

,125,74,1,110,99,33,21,38,107,116,...,65,15,120,121,68,95,32,124,131,corr
pIC50,5.958000,5.854000,6.292000,7.921000,5.854000,8.638000,7.301000,8.658000,5.658000,7.075000,...,5.34700,6.652000,7.221000,7.796000,6.180000,5.585000,8.699000,7.229000,7.259000,1.000000
VCH-5,0.079057,0.047140,0.000000,0.038490,0.047140,0.091287,0.000000,0.193349,0.047140,0.033333,...,0.00000,0.000000,0.079057,0.079057,0.000000,0.047140,0.091287,0.079057,0.020412,0.604421
C2SP3,5.000000,5.000000,4.000000,5.000000,5.000000,6.000000,4.000000,9.000000,6.000000,5.000000,...,3.00000,4.000000,5.000000,5.000000,3.000000,6.000000,6.000000,5.000000,5.000000,0.554315
SHCsatu,0.748232,2.428692,1.449385,0.756321,2.472921,0.733274,0.736191,1.234789,3.454298,0.701451,...,0.69061,0.733549,0.694354,0.748404,1.407289,3.454298,0.731087,0.758404,0.802897,0.549456
MATS6s,-0.073039,-0.157112,-0.075440,-0.060055,-0.100758,-0.044370,0.009816,-0.037839,-0.107407,-0.067171,...,-0.10379,-0.045215,-0.087945,-0.075164,-0.075538,-0.100230,-0.016963,-0.067262,-0.061281,0.544128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
nT4HeteroRing,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
nT8HeteroRing,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
nT10HeteroRing,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
nT12HeteroRing,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [25]:
corr_top100 = df_corr.iloc[1:101,-1]
corr_desc100 = corr_top100.index
corr_desc100

Index(['VCH-5', 'C2SP3', 'SHCsatu', 'MATS6s', 'VCH-6', 'nHCsatu', 'MATS6e',
       'minHdsCH', 'mindsCH', 'minHCsats', 'GATS6e', 'mindsN', 'maxdsN',
       'maxHCsatu', 'maxHdsCH', 'maxssCH2', 'ndsN', 'nHCsats', 'maxdsCH',
       'RDF30i', 'MATS6m', 'minssssC', 'MATS8i', 'SssssC', 'nHBAcc', 'MDEN-23',
       'SdsN', 'AATSC6e', 'nssssC', 'SpMax2_Bhs', 'RDF30u', 'MATS2i', 'ATSC6e',
       'SpMax1_Bhs', 'nHBAcc_Lipinski', 'GATS4i', 'AATSC6m', 'AATSC8i',
       'GATS2i', 'RDF30e', 'HybRatio', 'ETA_EtaP_F_L', 'AATSC2i', 'GATS2s',
       'ETA_EtaP_L', 'maxHCsats', 'ATSC6m', 'GATS6s', 'AVP-7', 'VCH-7',
       'AVP-4', 'GATS6m', 'SpMax3_Bhs', 'ATSC2i', 'AATSC6s', 'ATSC6s',
       'AATS3v', 'GATS2e', 'VCH-4', 'TDB9v', 'ATSC8i', 'AVP-6', 'nO',
       'SHBint7', 'WTPT-4', 'SCH-5', 'minHsNH2', 'AATSC3p', 'MATS3p', 'AVP-5',
       'SCH-4', 'GATS4p', 'TDB9m', 'maxHBint4', 'SRW5', 'MDEC-34', 'ATSC3p',
       'RDF30s', 'LOBMIN', 'ATSC2s', 'MATS2s', 'SpMax7_Bhs', 'minHBint4',
       'maxHBint5', 'n3Rin

In [26]:
pickle.dump(corr_desc100, open('data/corr_desc100.p','wb'))